In [1]:
# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "10g")
    .getOrCreate()
)


NameError: name 'SparkSession' is not defined

In [ ]:
tbl_merchants = spark.read.parquet("../data/tables/tbl_merchants.parquet")

In [ ]:
from pyspark.sql.functions import *
tbl_merchants = tbl_merchants.toPandas()

In [ ]:
# Observe strings format
tbl_merchants["tags"][0]

In [ ]:
# Separate tags from revenue and take rate
tags1 = tbl_merchants["tags"].str.split("\), ", expand=True)
tags2 = tags[0].str.split("\], ", expand=True)

In [ ]:
# Remove symbols from tag and making everything lowercase
tags3 = tags2[0].str.replace('[^\w\s]', '', regex = True)
tags4 = tags3.str.lower()

In [ ]:
tags4

In [ ]:
# https://relatedwords.io/furniture
furniture = ["house", "table", "room", "wood", "floor", "chair", "cabinet", \
             "items", "goods", "oak", "paintings", "renaissance", "bed", \
             "clothing", "kitchen", "clothes", "furniture", "floors", "baroque", \
             "decorations", "carpet", "toys", "pottery", "americana", "accessories", \
             "jewelry", "textiles", "lamp", "article of furniture", "neolithic", \
             "couch", "tile", "floor coverings", "appliances", "cabinets", \
             "antiques", "furnishings", "fabrics", "decor", "decorating", "carpets", \
             "flooring", "bedding", "furnishing", "mattress", "woodworking", "rugs", \
             "upholstery", "glassware", "wood flooring"]

In [ ]:
# https://relatedwords.io/electronics
electronics = ["games", "game", "video", "radio", "technology", "parts", "digital", \
               "entertainment", "internet", "car", "engineering", "security", "computer", \
               "metal", "electric", "equipment", "programming", "photo", "electronic", \
               "audio", "software", "motor", "mobile", "tech", "physics", "device", "phone", \
               "mechanical", "electricals", "lights", "electrical", "devices", "electronics", \
               "electricity", "battery", "computers", "hardware", "mechanics", "clothes", \
               "furniture", "gaming", "automotive", "micro", "wireless", "aerospace", \
               "automated", "toys", "digital cameras", "batteries", "stereos"]

In [ ]:
# https://relatedwords.io/fashion
fashion = ["style", "clothing", "outfit", "lifestyle", "beauty", "vogue", "haute couture", \
           "trend", "tailor", "manner", "couture", "designer", "styles", "dresses", "retro", \
           "mode", "makeup", "cut", "footwear", "designers", "fashion", "lingerie", "way", \
           "fit", "make", "clothes", "art", "design", "forge", "menswear", "stylist", \
           "fashion designer", "women wear", "form", "aesthetic", "pattern", "society", \
           "lanvin", "music", "fashionistas", "catwalks", "editorial", "language", "brand", \
           "versace", "dress", "michael kors", "paris fashion week", "fashionably" ,"vivienne westwood"]

In [ ]:
# https://relatedwords.io/beauty-products
beauty = ["perfume", "beauty", "hair", "lipstick", "rouge", "beautician", \
          "food and drug administration", "cream", "lotion", "fine", "grace", \
          "lovely", "product", "art", "mascara", "scent", "hairspray", "shampoo"]
# https://relatedwords.io/health
health = ["family", "improve", "education", "research", "social", "health", \
          "families", "medical", "food", "hospital", "care", "medicine", "agriculture", "safety"]
# https://relatedwords.io/household
household = ["people", "family", "life", "living", "personal", "work", "home", \
             "house", "community", "business", "person", "close", "domestic", \
             "child", "individual", "food", "parents", "car"]

beauty_health_personal_household = beauty + health + household

In [ ]:
# https://relatedwords.io/hobby
hobby = ["sports", "art", "game", "sport", "interest", "amateur", "photography", "fun", \
         "recreation", "profession", "hunting", "passion", "craft", "pursuit", "lifestyle", \
         "recreational", "genealogy", "leisure"]
# https://relatedwords.io/toys
toys = ["play", "outfits", "doll", "lego", "plastic", "meccano", "dally", "dolls", \
        "ball", "candy", "miniature", "yo-yo", "barbie", "hobby", "teddy bear", "baby", "dollhouse"]
# https://relatedwords.io/diy
DIY = ["repair", "art", "ikea", "gardening", "mod", "do it yourself", "repairing", \
       "tlc", "repairs", "soho", "trug", "duluth", "cooke", "faq", "adirondack"]

toys_hobby_DIY = hobby + toys + DIY

In [ ]:
# https://www.shopify.com/au/blog/top-online-shopping-categories
categories = [fashion, furniture, electronics, beauty_health_personal_household, toys_hobby_DIY]
categories_label = ["fashion", "furniture", "electronics", "beauty, health, personal and household", "toys, hobbies and DIY"]

In [ ]:
import builtins as p
for i in range(0, len(tags4)):
    counts = [0]*len(categories)
    for j in range(0, len(categories)):
        for word in categories[j]:
            if word in tags4[i]:
                counts[j] += 1
    if counts == [0]*len(categories):
        tags4[i] = "other"
    else:   
        max_value = p.max(counts)
        max_index = counts.index(max_value)
        tags4[i] = categories_label[max_index]

In [ ]:
tags4

In [ ]:
type(tags4)

In [ ]:
tbl_merchants["categoriesed_tags"] = tags4

In [ ]:
tbl_merchants.head(20)